In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False)
from sklearn import neural_network as nn
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict, KFold
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("tox24_challenge_test_all_ready.csv.gz", compression="gzip")
df

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


,Unnamed: 0,Standardized_SMILES
0,0,C=C1CCC2CC1C2(C)C
1,1,COC(=O)C(C)Oc1ccc(Oc2ccc(Cl)cc2Cl)cc1
2,2,CCCCCCCCCCCCCC(=O)OCC(O)CO
3,3,CC(=O)OC1CC2CCC1(C)C2(C)C
4,4,CCCCCCCCCC=CCc1cc(O)oc1O
...,...,...
495,495,CCCCCCOC(=O)CCCCC(=O)OCCCCCC
496,496,CCCCCCOC(=O)c1ccccc1C(=O)OCCCCCC
497,497,C=CC(=C)CCCC(C)(C)O
498,498,CC(C)COC(=O)CCCCC(=O)OCC(C)C


In [2]:
# Display basic information and first few rows of the dataset
df.info(), df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           500 non-null    int64 
 1   Standardized_SMILES  500 non-null    object
dtypes: int64(1), object(1)
memory usage: 7.9+ KB


(None,
    Unnamed: 0                    Standardized_SMILES
 0           0                      C=C1CCC2CC1C2(C)C
 1           1  COC(=O)C(C)Oc1ccc(Oc2ccc(Cl)cc2Cl)cc1
 2           2             CCCCCCCCCCCCCC(=O)OCC(O)CO
 3           3              CC(=O)OC1CC2CCC1(C)C2(C)C
 4           4               CCCCCCCCCC=CCc1cc(O)oc1O)

In [3]:
# Generate Morgan Fingerprints
def generate_fingerprints(smiles, radius=2, nBits=2048):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        return np.array(AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits))
    else:
        return np.zeros((nBits,))

df['Fingerprints'] = df['Standardized_SMILES'].apply(generate_fingerprints)



In [4]:
df

,Unnamed: 0,Standardized_SMILES,Fingerprints
0,0,C=C1CCC2CC1C2(C)C,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,COC(=O)C(C)Oc1ccc(Oc2ccc(Cl)cc2Cl)cc1,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,CCCCCCCCCCCCCC(=O)OCC(O)CO,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,CC(=O)OC1CC2CCC1(C)C2(C)C,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,CCCCCCCCCC=CCc1cc(O)oc1O,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
495,495,CCCCCCOC(=O)CCCCC(=O)OCCCCCC,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
496,496,CCCCCCOC(=O)c1ccccc1C(=O)OCCCCCC,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
497,497,C=CC(=C)CCCC(C)(C)O,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
498,498,CC(C)COC(=O)CCCCC(=O)OCC(C)C,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [5]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt
import pickle


# Prepare features and target
X = np.array(list(df['Fingerprints']))
smiles = df['Standardized_SMILES'].values
    
test_df=pd.DataFrame()
    
predictions_all = []  # List to store all predictions

smiles_all = []       # List to store SMILES for each test sample
 
                                   
X_test = X

smiles_test = smiles

with open(f'models/model_batch_9.pkl', 'rb') as file:
    model = pickle.load(file)

y_preds = model.predict(X_test)

       
test_df=pd.DataFrame({
    'y_preds': y_preds})
test_df      

,y_preds
0,17.680141
1,66.447517
2,31.625124
3,12.742518
4,93.636635
...,...
495,21.943390
496,29.487017
497,22.790966
498,28.948664


In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   y_preds  500 non-null    float32
dtypes: float32(1)
memory usage: 2.1 KB


In [7]:
test_df.to_csv("output/external_test_preds.csv", index=False)